In [22]:
import os
import pandas as pd
import fitz ##PyMuPDF
import glob
import copy
from pprint import pprint

In [23]:
pdf_dir = '../pdf/*.pdf'

In [24]:
g = glob.glob( pdf_dir )
fn = g[16]
len(g),g[:3],fn

(38,
 ['../pdf\\CWC2018-19_Leg1_ResultsBook.pdf',
  '../pdf\\CWC2018-19_Leg2_ResultsBook.pdf',
  '../pdf\\CWC2018-19_Leg3_ResultsBook.pdf'],
 '../pdf\\OWG2018_ResultsBook.pdf')

In [25]:
## ラストストーンをどちらが持っているか?
def solve_laststone(df):
    for i , r in df.iterrows():
        if i == 0:
            continue
        target_value = 0.0
        s = df.loc[df.index == i - 1]    
        if ( (s.red.values[0] == 0) & (s.yellow.values[0] == 0) ): ## ZeroZeroならば、動かない
            target_value = s.last_stone_is_red.values[0]
        else: ## 得点が入れば、点を失ったもの後攻＝　赤の点数が黄色より大きい場合　黄色が後攻
            if (s.red.values[0] < s.yellow.values[0] ):
                target_value = True
            else:
                target_value = False
        df.loc[df.index == i,'last_stone_is_red']= target_value
    return df

def pageToDataFrame(page,num):
    tabs = page.find_tables()
    if len(tabs.tables) <= 0: return False ,None
    ## 1つ目のtableを作る = > Hanmerがわかる表を探索
    def GetScore( tabs ):
        for qtable in tabs.tables:
            scoreTable = qtable.extract()
            df = pd.DataFrame(scoreTable)
            if False == (len(df) == 2) : return False  , None 
            is_redHammer = (df[0].values == ['*','']).all()
            is_YellowHammer = (df[0].values == ['','*']).all()
            #print(df)
            if any([is_redHammer,is_YellowHammer]):
                return True , df
            pass
        return False  , None
    isFind , df = GetScore(tabs)
    if False == isFind:
        print('No Score',num )
        return None
    is_redHammer = (df[0].values == ['*','']).all()
    is_YellowHammer = (df[0].values == ['','*']).all()
    df = df.drop(0,axis=1)
    
    colname = df.columns.to_list()
    colname = list(map( str , colname))
    colname = colname[:-1] + ['Total']
    df.columns = colname
    df = df.T
    df = df.reset_index()
    df.columns = ['ends','red','yellow']

    def to_number(v):
        if v.isdecimal():return int(v)
        else:return None
    df['red'] = df['red'].map( to_number )
    df['yellow'] = df['yellow'].map( to_number )
    #df = df.dropna(subset=['red','yellow'],axis=0)
    df = df.astype( {'red':float,'yellow':float})

    df['red_diff'] = df['red']
    df.loc[df['yellow'] > 0,'red_diff'] = df.loc[df['yellow'] > 0,'yellow'] * -1
    ## 残end数
    index_v = df.index.to_list()
    df['remain_ends'] = [8-int(v) for v in index_v[:-1]] + [0]
    
    df.loc[df['remain_ends'] < 0 , 'remain_ends'] = 0
    assert(any([is_redHammer,is_YellowHammer]))
    df['last_stone_is_red'] = False
    df.loc[ df.index == 0,'last_stone_is_red' ] = is_redHammer
    df = solve_laststone(df)
    df['page'] = num
    return df


In [26]:
def keyPosion( test_array , key ):
    if not key in test_array: return -1
    else: return test_array.index(key)

In [27]:
page_list = []
doc = fitz.open(fn)  
for i ,page in enumerate(doc):   
    if i < 5 : continue
    text = page.get_text()
    q = text.split('\n')
    key = 'Game Results'  
    title_index = keyPosion( q , key )
    if title_index < 0 :pass
    else:
        page_list.append(i)
##page_list

In [28]:
for i in page_list:
    page = doc[i]
    break
pageToDataFrame(page, i)

,ends,red,yellow,red_diff,remain_ends,last_stone_is_red,page
0,1,0.0,1.0,-1.0,8,False,21
1,2,0.0,0.0,0.0,7,True,21
2,3,2.0,0.0,2.0,6,True,21
3,4,1.0,0.0,1.0,5,False,21
4,5,1.0,0.0,1.0,4,False,21
5,6,0.0,1.0,-1.0,3,False,21
6,7,3.0,0.0,3.0,2,True,21
7,8,0.0,1.0,-1.0,1,False,21
8,9,1.0,0.0,1.0,0,True,21
9,10,NaN,NaN,NaN,0,False,21


In [29]:
##l = [pageToDataFrame( doc[i] ) for i in page_list]
l = []
for i in page_list:
    print(i)
    l.append( pageToDataFrame( doc[i], i )  )

21
37
53
65
93
104
114
125
136
143
154
165
179
190
201
215
226
237
246
260
271
279
290
301
313
324
336
346
357
368
379
390
402
414
428
440
451
466
478
490
501
515
526
536
548
562
572
580
589
605
622
638
647
660
691
699
708
715
727
737
748
762
770
780
791
804
815
823
835
849
858
870
882
895
906
914
928
938
949
960
974
985
996
1006
1020
1031
1042
1043
No Score 1043
1055
1066
1077
1088
1102
1113
1123
1134
1148
1159
1170
1181
1200
1213
1227
1236
1248
1275
1282
1291
1300
1312
1319
1328
1335
1346
1355
1365
1372
1385
1392
1402
1409
1421
1430
1439
1447
1459
1468
1478
1487
1499
1507
1516
1524


In [9]:
page = doc[1043]
tabs = page.find_tables()
len(tabs.tables)
s = tabs.tables[1].extract()
display(s)
q = pd.DataFrame(s)
len(q) == 2

[['Note:\nPosition and function for each team member:\n4 = Fourth, 3 = Third, 2 = Second, 1 = Lead, A = Alternate, S = Skip, V = Vice-Skip'],
 [''],
 ['Legend:\nClockwise Counter-clockwise LSFE(*) Last Stone First End\nX Unplayed/unfinished end due to concession']]

False

In [10]:
page_list[:10]

[21, 37, 53, 65, 93, 104, 114, 125, 136, 143]

In [11]:
#num = page_list.index(330)
#num = page_list.index(224)
num = page_list.index(page_list[-1])

dx = l[num] 
dx

,ends,red,yellow,red_diff,remain_ends,last_stone_is_red,page
0,1,1.0,0.0,1.0,8,True,1524
1,2,0.0,1.0,-1.0,7,False,1524
2,3,0.0,1.0,-1.0,6,True,1524
3,4,2.0,0.0,2.0,5,True,1524
4,5,0.0,1.0,-1.0,4,False,1524
5,6,1.0,0.0,1.0,3,True,1524
6,7,0.0,2.0,-2.0,2,False,1524
7,8,1.0,0.0,1.0,1,True,1524
8,9,0.0,2.0,-2.0,0,False,1524
9,10,NaN,NaN,NaN,0,True,1524


## 画像をあつめる

In [12]:
outpath = './img/'
def SaveImage( doc ,outpath , game , ends ):
    if ends == 'Total' : 
        return ""
    ends = int(ends)
    page = doc[game+ends]
    text = page.get_text()
    text_array = text.split('\n')
    index = keyPosion(text_array,'Game - Shot by Shot')
    #print(index, index < 0)
    if index < 0:
        print(game,ends,'game error : no Shot by Shot')
        return ""
    assert( index > 0 )
    for text in text_array:
        if 'End ' in text:
            page_ends = int(text.split(' ')[-1])
            #print(page_ends,ends,text.split(' ')[-1] )
            if (page_ends == ends):
                pass
            else:
                print(game,ends,'ends error : no Shot by Shot')
                return ""
            break
    
    cimageList = page.get_images()
    dq = pd.DataFrame(cimageList)
    dq.to_csv('imgList.csv')
    def IndexFinder(dq):
        
        dqz = dq.loc[ (dq[5] == 'Indexed') & (dq[3] > 550) & (dq[3] < 610)].copy()
        dqz['sort'] = dq[7].map(lambda s:int(s[2:]))
        if False == (len(dqz) == 16): return -1 , 'NG'
        dqz = dqz.sort_values('sort')
        print(dqz)
        target_s = dqz.iloc[14]
        return target_s.iloc[0] , target_s.iloc[7]
    imageId , name = IndexFinder(dq)
    print(imageId , type(imageId) , name )
    if imageId < 0:
        print(f'game {game}-{ends} no image')
        return ""
    imageId = int(imageId)
    img = doc.extract_image(imageId)
    ext = img['ext']
    path = os.path.join(outpath,f'geme{game}end{ends}.{ext}')
    with open(path, "wb") as ofh:
        ofh.write(img['image'])
    return os.path.basename(path)
    
    

In [13]:
dq = pd.concat(l)
dq = dq.loc[dq['page'] < 100]
dq[:10]

,ends,red,yellow,red_diff,remain_ends,last_stone_is_red,page
0,1,0.0,1.0,-1.0,8,False,21
1,2,0.0,0.0,0.0,7,True,21
2,3,2.0,0.0,2.0,6,True,21
3,4,1.0,0.0,1.0,5,False,21
4,5,1.0,0.0,1.0,4,False,21
5,6,0.0,1.0,-1.0,3,False,21
6,7,3.0,0.0,3.0,2,True,21
7,8,0.0,1.0,-1.0,1,False,21
8,9,1.0,0.0,1.0,0,True,21
9,10,NaN,NaN,NaN,0,False,21


In [14]:
dq = dq.reset_index(drop = True)
dq['filepath'] = dq[:20].apply(lambda r: SaveImage( doc, outpath , r.page , r.ends ) , axis=1)

     0  1    2    3  4        5 6     7            8  sort
18  88  0  300  600  4  Indexed     Im4  FlateDecode     4
19  89  0  300  600  4  Indexed     Im5  FlateDecode     5
20  90  0  300  600  4  Indexed     Im6  FlateDecode     6
21  91  0  300  600  4  Indexed     Im7  FlateDecode     7
22  92  0  300  600  4  Indexed     Im8  FlateDecode     8
23  93  0  300  600  4  Indexed     Im9  FlateDecode     9
2   78  0  300  600  4  Indexed    Im10  FlateDecode    10
3   79  0  300  600  4  Indexed    Im11  FlateDecode    11
4   80  0  300  600  4  Indexed    Im12  FlateDecode    12
5   81  0  300  600  4  Indexed    Im13  FlateDecode    13
6   82  0  300  600  4  Indexed    Im14  FlateDecode    14
7   83  0  300  600  4  Indexed    Im15  FlateDecode    15
8   84  0  300  600  4  Indexed    Im16  FlateDecode    16
9   85  0  300  600  4  Indexed    Im17  FlateDecode    17
10  86  0  300  600  4  Indexed    Im18  FlateDecode    18
11  87  0  300  600  4  Indexed    Im19  FlateDecode    

In [15]:
print(fn)
dq[:20]

../pdf\OWG2018_ResultsBook.pdf


,ends,red,yellow,red_diff,remain_ends,last_stone_is_red,page,filepath
0,1,0.0,1.0,-1.0,8,False,21,geme21end1.png
1,2,0.0,0.0,0.0,7,True,21,geme21end2.png
2,3,2.0,0.0,2.0,6,True,21,geme21end3.png
3,4,1.0,0.0,1.0,5,False,21,geme21end4.png
4,5,1.0,0.0,1.0,4,False,21,geme21end5.png
5,6,0.0,1.0,-1.0,3,False,21,geme21end6.png
6,7,3.0,0.0,3.0,2,True,21,geme21end7.png
7,8,0.0,1.0,-1.0,1,False,21,geme21end8.png
8,9,1.0,0.0,1.0,0,True,21,geme21end9.png
9,10,NaN,NaN,NaN,0,False,21,


In [16]:
fn

'../pdf\\OWG2018_ResultsBook.pdf'

In [17]:
dq = pd.read_csv('imgList.csv',index_col=0)
z = dq['7'].map(lambda s:int(s[2:]))
z

0      0
1      1
2     10
3     11
4     12
5     13
6     14
7     15
8     16
9     17
10    18
11    19
12     2
13    20
14    21
15    22
16    23
17     3
18     4
19     5
20     6
21     7
22     8
23     9
Name: 7, dtype: int64

In [18]:

dqz = dq.loc[ (dq['5'] == 'Indexed') & (dq['3'] > 550) & (dq['3'] < 610)].copy()
dqz['sort'] = dq['7'].map(lambda s:int(s[2:]))
if False == (len(dqz) == 16): print( -1 , 'NG' )
target_s = dqz.sort_values('sort').iloc[15]
print( target_s.iloc[0] , target_s.iloc[7])
dqz.sort_values('0').iloc[14]

379 Im19


0               385
1                 0
2               300
3               600
4                 4
5           Indexed
6               NaN
7               Im8
8       FlateDecode
sort              8
Name: 22, dtype: object

In [19]:

def IndexFinder(dq):
    dqz = dq.loc[ (dq['5'] == 'Indexed') & (dq['3'] > 550) & (dq['3'] < 610)]
    if False == (len(dqz) == 16): return -1 , 'NG'
    target_s = dqz.sort_values('0').iloc[14]
    return target_s.iloc[0] , target_s.iloc[7]
t , name = IndexFinder (dq)
t

385

In [20]:
t

385

In [21]:
ext = img['ext']
outpath = './img/'
path = os.path.join(outpath,f'geme{game}end{ends}.{ext}')
with open(path, "wb") as ofh:
    ofh.write(img['image'])


NameError: name 'img' is not defined

In [ ]:
colname = df.columns.to_list()

In [ ]:
colname = list(map( str , colname))

In [ ]:
colname = colname[:-1] + ['Total']

In [ ]:
colname

In [ ]:
df.columns = colname
df

In [ ]:
df = df.T

In [ ]:
df['isRedLastStone'] = False

In [ ]:
df

In [ ]:
page_i = 330
page_i = 231

In [ ]:
page = doc[page_i]
tabs = page.find_tables()
##if len(tabs.tables) <= 0: return None
print(len(tabs.tables))
## 1つ目のtableを作る
scoreTable = tabs.tables[0].extract()
df = pd.DataFrame(scoreTable)
is_redHammer = (df[0].values == ['*','']).all()
is_YellowHammer = (df[0].values == ['','*']).all()
assert(any([is_redHammer,is_YellowHammer]))
df = df.drop(0,axis=1)

colname = df.columns.to_list()
colname = list(map( str , colname))
colname = colname[:-1] + ['Total']
df.columns = colname

In [ ]:
df0 = df.copy()
df = df.T
df = df.reset_index()
df.columns = ['ends','red','yellow']
#df

In [ ]:
scoreTable

In [ ]:
df = pd.DataFrame(scoreTable)
is_redHammer = (df[0].values == ['*','']).all()
is_YellowHammer = (df[0].values == ['','*']).all()
is_redHammer , is_YellowHammer


In [ ]:
df = df.astype( {'red':int,'yellow':int})

df['red_diff'] = df['red']
df.loc[df['yellow'] > 0,'red_diff'] = df.loc[df['yellow'] > 0,'yellow'] * -1
## 残end数
index_v = df.index.to_list()
df['remain_ends'] = [8-int(v) for v in index_v[:-1]] + [0]

df.loc[df['remain_ends'] < 0 , 'remain_ends'] = 0

In [ ]:
df

In [ ]:
assert(any([is_redHammer,is_YellowHammer]))
df['last_stone_is_red'] = False
df.loc[ df.index == 0,'last_stone_is_red' ] = is_redHammer
df

In [ ]:
pwd

In [ ]:
df

In [ ]:
scoreTable

In [ ]:
## チームが取得した点数　or 失った点数
dqw['points'] = dqw['Y']
dqw.loc[ dqw['points'] == 0, 'points'] = dqw['R']  * -1
## 残end数
dqw['remain_ends'] = 10-dqw['ends'] 
dqw.loc[ dqw['remain_ends'] < 0 ,'remain_ends'] = 0
## ラストストーンをどちらが持っているか?
for i , r in dqw.iterrows():
    if i == 0:
        continue
    target_value = 0.0
    s = dqw.loc[dqw['ends'] == (r.ends - 1)]    
    if ( (s.Y.values[0] == 0) & (s.R.values[0] == 0) ): ## ZeroZeroならば、動かない
        target_value = s.last_stone_is_yellow.values[0]
    else: ## 得点が入れば、点を失ったもの後攻＝　赤の点数が黄色より大きい場合　黄色が後攻
        if (s.Y.values[0] < s.R.values[0] ):
            target_value = 1.0
        else:
            target_value = 0.0
    dqw.loc[dqw.index == i,'last_stone_is_yellow']= target_value